In [332]:
import pandas as pd 
import numpy as np
import re


In [361]:
data = pd.read_excel('DATA PASIEN POLI GIGI 2023-2024.xlsx', dtype={'Nomor Hp': str})


# drop all columns contain word 'Unnamed'
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data.head(5)

,NO,TANGGAL,NOMOR REKAM MEDIS,NAMA PASIEN,ALAMAT LENGKAP,Nomor Hp,NO KK,NO NIK,TGL LAHIR,NAMA KEPALA KELUARGA,...,Balita (1-5 thn),Anak (6-18 thn),Dewasa (19-59 thn),Lansia (≥ 60 thn),KELUHAN,DIAGNOSA,TERAPI OBAT,UMUM/ GIGI,MEMBER,NON MEMBER
0,1,2023-01-03 00:00:00,1057MDN011964,Tn Suparno,Desa Kanung 07/01,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,v,NaN,sakit gigi,abses gingiva,"clindamycin 3x1, piroxicam 2x1",P GIGI,NaN,Rp -
1,2,2023-01-03 00:00:00,1033MDN011990,Tn Roikhan,Ds Kanung 05/02,82229436214,NaN,NaN,NaN,NaN,...,NaN,NaN,v,NaN,kontrol gigi,GMK + pulpitis ireversible,scaling,P GIGI,NaN,Rp -
2,3,2022-01-05 00:00:00,0704MDN011988,Ny.Ely Ernawati,Desa rejosari RT/RW 02/01,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,v,NaN,Scaling,GMK,scaling,P GIGI,NaN,60000
3,4,2023-01-09 00:00:00,0699MDN012014,Ank.Deafi Septina,Desa rejosari RT/RW 1/1,08581552550060,NaN,NaN,NaN,NaN,...,NaN,v,NaN,NaN,"gigi goyang, cabut gigi",persistensi,exo ce,P GIGI,NaN,Rp -
4,5,2023-01-12 00:00:00,1057MDN091982,Ny. Siti Amirul,Rejosari 7/2,081234377794,NaN,NaN,NaN,NaN,...,NaN,NaN,V,NaN,"Lepas tambalan, scaling, grinding","pulpitis irversible, GMK",Scaling,P GIGI,NaN,90000


In [362]:
# rename column 'MEMBER' to 'MEMBER FEE', 'NON MEMBER' to 'NON MEMBER FEE', ' MEMBER' to 'MEMBER' and 'NON MEMBER ' to 'NON MEMBER'
data = data.rename(columns={'MEMBER': 'MEMBER FEE', 'NON MEMBER': 'NON MEMBER FEE', ' MEMBER': 'MEMBER', 'NON MEMBER ': 'NON MEMBER'})

In [363]:
pronouns = ['Tn', 'Ny', 'Ank', 'An', 'Sdr']  # List of pronouns
pattern = rf"^({'|'.join([re.escape(pronoun) for pronoun in pronouns])})\.?\s*"

data['NAMA PASIEN'] = data['NAMA PASIEN'].str.replace(pattern, '', regex=True)
data['NAMA PASIEN'] = data['NAMA PASIEN'].str.lower()

data['NAMA KEPALA KELUARGA'] = data['NAMA KEPALA KELUARGA'].str.replace(pattern, '', regex=True)
data['NAMA KEPALA KELUARGA'] = data['NAMA KEPALA KELUARGA'].str.lower()


In [364]:
data['NAMA PASIEN'].value_counts()

NAMA PASIEN
hafid zamrudin             10
naufal rafi hamizan         9
elmira azzahra nugraha      9
srinatun                    9
albi s                      8
                           ..
sumadi                      1
yuli rosalia                1
hanifa cahya wardati        1
afzahra fentiara            1
dhinar anzani adevilyon     1
Name: count, Length: 259, dtype: int64

In [365]:
data['Member Remark'] = data.apply(lambda row: 1 if row['MEMBER'] == 'MEMBER' or row['NON MEMBER'] == 'MEMBER' else 0 if row['MEMBER'] == 'NON MEMBER' or row['NON MEMBER'] == 'NON MEMBER' else None, axis=1).astype('Int64')

In [366]:
data['Member Remark'].value_counts()

Member Remark
0    379
1    122
Name: count, dtype: Int64

In [367]:
data.columns

Index(['NO', 'TANGGAL', 'NOMOR REKAM MEDIS', 'NAMA PASIEN', 'ALAMAT LENGKAP',
       'Nomor Hp', 'NO KK', 'NO NIK', 'TGL LAHIR', 'NAMA KEPALA KELUARGA',
       'MEMBER', 'NON MEMBER', 'L', 'P', 'Bayi (0-11 bln)', 'Balita (1-5 thn)',
       'Anak (6-18 thn)', 'Dewasa (19-59 thn)', 'Lansia (≥ 60 thn)', 'KELUHAN',
       'DIAGNOSA', 'TERAPI OBAT', 'UMUM/ GIGI', 'MEMBER FEE', 'NON MEMBER FEE',
       'Member Remark'],
      dtype='object')

In [368]:
data['Age Category'] = data[['Bayi (0-11 bln)', 'Balita (1-5 thn)', 'Anak (6-18 thn)', 'Dewasa (19-59 thn)','Lansia (≥ 60 thn)']].apply(lambda row: row.first_valid_index(), axis=1)


In [369]:
# drop column 'Bayi (0-11 bln)', 'Balita (1-5 thn)', 'Anak (6-18 thn)', 'Dewasa (19-59 thn)','Lansia (≥ 60 thn)'
data = data.drop(columns=['Bayi (0-11 bln)', 'Balita (1-5 thn)', 'Anak (6-18 thn)', 'Dewasa (19-59 thn)','Lansia (≥ 60 thn)'])


In [370]:
data['Age'] = data[['L', 'P']].bfill(axis=1).iloc[:, 0]

data['Gender'] = data.apply(lambda row: 'L' if pd.notna(row['L']) else 'P' if pd.notna(row['P']) else None, axis=1)



In [371]:
data.columns

Index(['NO', 'TANGGAL', 'NOMOR REKAM MEDIS', 'NAMA PASIEN', 'ALAMAT LENGKAP',
       'Nomor Hp', 'NO KK', 'NO NIK', 'TGL LAHIR', 'NAMA KEPALA KELUARGA',
       'MEMBER', 'NON MEMBER', 'L', 'P', 'KELUHAN', 'DIAGNOSA', 'TERAPI OBAT',
       'UMUM/ GIGI', 'MEMBER FEE', 'NON MEMBER FEE', 'Member Remark',
       'Age Category', 'Age', 'Gender'],
      dtype='object')

In [372]:
data = data.drop(columns=['L', 
                          'P',
                          'NO KK',
                          'NO NIK',
                          'NOMOR REKAM MEDIS',
                          ])

In [373]:
# Define the mapping of keywords to categories
categories = {
    'cabut gigi': ['cabut gigi', 'cabut', 'lepas'],
    'kontrol gigi': ['kontrol', 'periksa', 'konsultasi', 'cek', 'kontro', 'konsul'],
    'sakit gigi': ['sakit', 'ngilu', 'nyeri'],
    'tambal gigi': ['tambal'],
    'scaling': ['scalling', 'sacling', 'scaling'],
    'gusi bengkak': ['bengkak'],
    'gigi berlubang': ['berlubang', 'lubang'],
    'karang gigi': ['karang'],
}

# Loop through the categories and update the 'KELUHAN' column
for category, keywords in categories.items():
    pattern = '|'.join(keywords)  # Join keywords into a regex pattern
    data.loc[data['KELUHAN'].str.contains(pattern, na=False, case=False), 'KELUHAN'] = category


In [374]:
data['KELUHAN'].value_counts().head(15)

KELUHAN
kontrol gigi                              106
sakit gigi                                 89
tambal gigi                                85
karang gigi                                71
cabut gigi                                 57
gigi berlubang                             38
gusi bengkak                               22
scaling                                    15
gigi goyang                                 6
gigi tumbuh                                 4
Gigi tumbuh                                 2
pasang gigi                                 1
mual                                        1
gigi goyang dan gigi baru sudah tumbuh      1
gigi palsu goyang                           1
Name: count, dtype: int64

In [375]:
keluhan_counts = data['KELUHAN'].value_counts()
filtered_keluhan_counts = keluhan_counts[keluhan_counts < 10]
filtered_keluhan_counts

KELUHAN
gigi goyang                                             6
gigi tumbuh                                             4
Gigi tumbuh                                             2
pasang gigi                                             1
mual                                                    1
gigi goyang dan gigi baru sudah tumbuh                  1
gigi palsu goyang                                       1
sariawan                                                1
tertimpa tangga, luka di dagu dan bibir, gigi goyang    1
gigi tajam                                              1
Gigi tajam sudah 1 minggu                               1
Gigi Tumbuh                                             1
Gigi Tajam                                              1
Gigi atas patah                                         1
TIDAK NYAMAN PADA GIGI BELAKANG                         1
Gigi kiri ,kanan belakang tumbuh dan tidak nyaman       1
Gigi Goyang                                             1
Name: 

In [376]:
# set 'lainnya' to column 'KELUHAN' where the value count is less than 10 
data.loc[data['KELUHAN'].isin(filtered_keluhan_counts.index), 'KELUHAN'] = 'lainnya'
data['KELUHAN'].value_counts()

KELUHAN
kontrol gigi      106
sakit gigi         89
tambal gigi        85
karang gigi        71
cabut gigi         57
gigi berlubang     38
lainnya            26
gusi bengkak       22
scaling            15
Name: count, dtype: int64

In [377]:
data['DIAGNOSA']

0                   abses gingiva
1      GMK + pulpitis ireversible
2                             GMK
3                     persistensi
4        pulpitis irversible, GMK
                  ...            
505         PULPITIS IRREVERSIBLE
506                 PERIODENTITIS
507         PULPITIS IRREVERSIBLE
508                           GMK
509         PULPITIS IRREVERSIBLE
Name: DIAGNOSA, Length: 510, dtype: object

In [378]:
data.to_csv('data_pasien.csv', index=False)

In [379]:
cleaned_diagnoses = ['48 impacted',
 'abs gingiva nekrosisi',
 'abses bukalis',
 'abses ggv ant mandibula',
 'abses gingiva',
 'abses gingiva e.c periodentitis kronis',
 'abses gingiva ec nekrosis pulpa',
 'cavit',
 'dental examination',
 'diastema interdental',
 'dry eyes',
 'edentolous',
 'edentolous ridge',
 'fracture dental',
 'fraktur dental',
 'fraktur lingual',
 'fraktur mahkota',
 'gangren pulpa',
 'gangren radi x',
 'gangren radix',
 'gangren radix ( nekrosis )',
 'gangren radix 23',
 'gingiva disease',
 'gingivitis',
 'gingivitis kronis',
 'glositis',
 'hiperplastik gingiva',
 'impaksi',
 'impaksi gigi',
 'moloklusi',
 'myalgia',
 'nekrosis',
 'nekrosis pulpa',
 'nekrosis pulpitis',
 'nekrosisi pulpa',
 'partial edentolosis ridge',
 'partial edentulous ridge',
 'partial erupted',
 'perikoronitis',
 'periodentitis',
 'periodentitis agresif',
 'periodentitis akut',
 'periodentitis apikalis akut',
 'periodentitis kronis',
 'periodontitis',
 'periokoronitis',
 'persistensi',
 'priodentitis kronis',
 'pulpitis',
 'pulpitis akut',
 'pulpitis ireversible',
 'pulpitis irresversible',
 'pulpitis irreversible',
 'pulpitis irversible',
 'pulpitis reversible',
 'pulpitis reversible scaling',
 'pulpitis rreversible',
 'pulpitits irreversible',
 'pulpitits reversible',
 'puplitis irreversible',
 'retained dental root',
 'stomatitis',
 'trauma oklusi',
 'traumatic ulcer',
 'ulcertiva stomatitis']

In [380]:
# # Function to split raw text in 'DIAGNOSA' into lists of cleaned terms
# def parse_and_clean_diagnosa(entry):
#     if pd.isna(entry):
#         return []
#     # Split on common delimiters like '+' or ','
#     items = re.split(r'[+,]', entry.lower())
#     # Clean and strip each item
#     return [item.strip() for item in items if len(item.strip()) > 0]

# # Recreate 'Cleaned DIAGNOSA' column
# data['Cleaned DIAGNOSA'] = data['DIAGNOSA'].apply(parse_and_clean_diagnosa)

# # Inspect the updated 'Cleaned DIAGNOSA' column
# cleaned_diagnosa_sample = data['Cleaned DIAGNOSA'].head()
# cleaned_diagnosa_sample

In [387]:
# Step 1: Extract unique diagnoses and clean them
def extract_and_clean_diagnoses(diagnoses):
    cleaned_diagnoses = set()
    for entry in diagnoses:
        if pd.isna(entry):
            continue
        # Normalize and split entries on common delimiters
        items = re.split(r'[+,]', entry.lower())
        for item in items:
            item = item.strip()
            if len(item) > 2:  # Basic filter for meaningful terms
                cleaned_diagnoses.add(item)
    return sorted(cleaned_diagnoses)

# Extract unique diagnoses from the 'DIAGNOSA' column
unique_diagnoses = data['DIAGNOSA'].dropna().unique()
cleaned_diagnoses = extract_and_clean_diagnoses(unique_diagnoses)

# Ensure 'gingivitis marginalis kronis' is included
cleaned_diagnoses.append('gingivitis marginalis kronis')
cleaned_diagnoses.append('abses gingiva')
cleaned_diagnoses.append('gingival abscess')

# Step 2: Define standardized terms mapping
standardized_terms = {
    # General diagnoses
    'pulpitis irreversible': 'pulpitis irreversible',
    'pulpitis ireversible': 'pulpitis irreversible',
    'puplitis irreversible': 'pulpitis irreversible',
    'pulpitits irreversible': 'pulpitis irreversible',
    'pulpitis reversible': 'pulpitis reversible',
    'pulpitits reversible': 'pulpitis reversible',
    'pulpitis rreversible': 'pulpitis reversible',
    'nekrosis': 'necrosis pulpa',
    'nekrosis pulpa': 'necrosis pulpa',
    'nekrosis pulpitis': 'necrosis pulpa',
    'nekrosisi pulpa': 'necrosis pulpa',

    # Gingivitis and related terms
    'gmk': 'gingivitis marginalis kronis',
    'gingivitis kronis': 'chronic gingivitis',
    'gingivitis': 'gingivitis',
    'hiperplastik gingiva': 'gingival hyperplasia',

    # Abscesses
    'abses gingiva': 'gingival abscess',
    'abses bukalis': 'buccal abscess',
    'abses ggv ant mandibula': 'gingival abscess anterior mandible',
    'abses gingiva ec nekrosis pulpa': 'gingival abscess with necrosis pulpa',

    # Fractures and trauma
    'fracture dental': 'dental fracture',
    'fraktur dental': 'dental fracture',
    'fraktur mahkota': 'crown fracture',
    'trauma oklusi': 'occlusal trauma',

    # Periodontal diseases
    'periodontitis': 'periodentitis',
    'periodentitis akut': 'acute periodentitis',
    'periodentitis kronis': 'chronic periodentitis',
    'priodentitis kronis': 'chronic periodentitis',
    'periodontitis apikalis akut': 'acute apical periodontitis',
    'periokoronitis': 'pericoronitis',

    # Other terms
    'edentolous ridge': 'edentulous ridge',
    'partial edentulous ridge': 'edentulous ridge',
    'diastema interdental': 'interdental diastema',
    'gangren radix': 'radicular gangrene',
    'stomatitis': 'stomatitis',
    'traumatic ulcer': 'traumatic ulcer',
    'ulcertiva stomatitis': 'ulcerative stomatitis',
}

# Step 3: Function to transform diagnoses into standardized lists
def transform_to_standardized_diagnoses(entry):
    if pd.isna(entry):
        return []
    # Normalize and split the text
    items = re.split(r'[+,]', entry.lower())
    standardized_items = []
    for item in items:
        item = item.strip()
        # Map to standardized terms or keep original if not in the mapping
        standardized_item = standardized_terms.get(item, item)
        if standardized_item in cleaned_diagnoses:
            standardized_items.append(standardized_item)
    return standardized_items

# Step 4: Apply transformation to create 'Cleaned DIAGNOSA'
data['Cleaned DIAGNOSA'] = data['DIAGNOSA'].apply(transform_to_standardized_diagnoses)




In [388]:
data

,NO,TANGGAL,NAMA PASIEN,ALAMAT LENGKAP,Nomor Hp,TGL LAHIR,NAMA KEPALA KELUARGA,MEMBER,NON MEMBER,KELUHAN,DIAGNOSA,TERAPI OBAT,UMUM/ GIGI,MEMBER FEE,NON MEMBER FEE,Member Remark,Age Category,Age,Gender,Cleaned DIAGNOSA
0,1,2023-01-03 00:00:00,suparno,Desa Kanung 07/01,NaN,NaN,NaN,MEMBER,NaN,sakit gigi,abses gingiva,"clindamycin 3x1, piroxicam 2x1",P GIGI,NaN,Rp -,1,Dewasa (19-59 thn),58.0,L,[gingival abscess]
1,2,2023-01-03 00:00:00,roikhan,Ds Kanung 05/02,82229436214,NaN,NaN,MEMBER,NaN,kontrol gigi,GMK + pulpitis ireversible,scaling,P GIGI,NaN,Rp -,1,Dewasa (19-59 thn),32.0,L,"[gingivitis marginalis kronis, pulpitis irreve..."
2,3,2022-01-05 00:00:00,ely ernawati,Desa rejosari RT/RW 02/01,NaN,NaN,NaN,NON MEMBER,NaN,scaling,GMK,scaling,P GIGI,NaN,60000,0,Dewasa (19-59 thn),34.0,P,[gingivitis marginalis kronis]
3,4,2023-01-09 00:00:00,deafi septina,Desa rejosari RT/RW 1/1,08581552550060,NaN,NaN,MEMBER,NaN,cabut gigi,persistensi,exo ce,P GIGI,NaN,Rp -,1,Anak (6-18 thn),15.0,P,[persistensi]
4,5,2023-01-12 00:00:00,siti amirul,Rejosari 7/2,081234377794,NaN,NaN,NaN,NON MEMBER,cabut gigi,"pulpitis irversible, GMK",Scaling,P GIGI,NaN,90000,0,Dewasa (19-59 thn),40.0,P,"[pulpitis irversible, gingivitis marginalis kr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,506,2024-12-23 00:00:00,elvivana maisaroh,Ds Pucangrejo 13/06,081359274517,1982-12-12 00:00:00,samsul riyadi,NaN,NON MEMBER,tambal gigi,PULPITIS IRREVERSIBLE,T LC,GIGI,NaN,60000,0,Dewasa (19-59 thn),42.0,P,[pulpitis irreversible]
506,507,2024-12-24 00:00:00,muhamad refan choirul iman,REJOSARI 07/02,NaN,2018-02-23 00:00:00,budianto,NaN,NON MEMBER,lainnya,PERIODENTITIS,"EXT TOPIKAL, CE",GIGI,NaN,30000,0,Anak (6-18 thn),6.0,L,[periodentitis]
507,508,2024-12-24 00:00:00,narti,Ds. Kanung 02/02,NaN,1966-06-06 00:00:00,saroni,MEMBER,NaN,kontrol gigi,PULPITIS IRREVERSIBLE,T LC,GIGI,0.0,NaN,1,Dewasa (19-59 thn),58.0,P,[pulpitis irreversible]
508,509,2024-12-24 00:00:00,dhinar anzani adevilyon,Karangmojo 13/4 Magetan,NaN,1997-05-26 00:00:00,mulyono,NaN,NON MEMBER,karang gigi,GMK,SCALING,GIGI,NaN,0,0,Dewasa (19-59 thn),27.0,P,[gingivitis marginalis kronis]


In [389]:
data['Cleaned DIAGNOSA'].isnull().sum()

0

In [390]:
standardized_terms_terapi = {
    'scaling': 'scaling procedure',
    'scalling': 'scaling procedure',
    'grinding': 'tooth grinding',
    'karang gigi': 'tartar removal',
    'lc': 'local anesthesia',
    'ts': 'temporary seal',
    'gic': 'glass ionomer cement',
    'evg': 'evacuate gingiva',
    'cavit': 'cavity treatment',
    'clindamycin 3x1': 'clindamycin (3x1 dosage)',
    'piroxicam 2x1': 'piroxicam (2x1 dosage)',
    'kaditic 2x1': 'kaditic (2x1 dosage)',
    'diclofenac 2x1': 'diclofenac potassium (2x1 dosage)',
    'amoxicilin 3x1': 'amoxicillin (3x1 dosage)',
    'asam mefenamat 1x1': 'mefenamic acid (1x1 dosage)',
    # Add more mappings based on context as necessary
}

In [391]:
# Step 1: Extract unique treatments and clean them
def extract_and_clean_treatments(treatments):
    cleaned_treatments = set()
    for entry in treatments:
        if pd.isna(entry):
            continue
        # Normalize and split entries on common delimiters
        items = re.split(r'[+,]', entry.lower())
        for item in items:
            item = item.strip()
            if len(item) > 2:  # Basic filter for meaningful terms
                cleaned_treatments.add(item)
    return sorted(cleaned_treatments)

# Extract unique treatments from the 'TERAPI OBAT' column
unique_treatments = data['TERAPI OBAT'].dropna().unique()
cleaned_treatments = extract_and_clean_treatments(unique_treatments)

# Ensure important terms are included
for term in standardized_terms_terapi.values():
    if term not in cleaned_treatments:
        cleaned_treatments.append(term)

# Step 2: Function to transform treatments into standardized lists
def transform_to_standardized_treatments(entry):
    if pd.isna(entry):
        return []
    # Normalize and split the text
    items = re.split(r'[+,]', entry.lower())
    standardized_items = []
    for item in items:
        item = item.strip()
        # Map to standardized terms or keep original if not in the mapping
        standardized_item = standardized_terms_terapi.get(item, item)
        if standardized_item in cleaned_treatments:
            standardized_items.append(standardized_item)
    return standardized_items

# Step 3: Apply transformation to create 'Cleaned TERAPI OBAT'
data['Cleaned TERAPI OBAT'] = data['TERAPI OBAT'].apply(transform_to_standardized_treatments)






In [392]:
data['Cleaned TERAPI OBAT'].isnull().sum()

0

In [393]:
from collections import Counter

# Flatten the lists for 'Cleaned DIAGNOSA'
all_diagnoses = [item for sublist in data['Cleaned DIAGNOSA'] for item in sublist]
diagnosis_counts = pd.Series(Counter(all_diagnoses))

# Flatten the lists for 'Cleaned TERAPI OBAT'
all_treatments = [item for sublist in data['Cleaned TERAPI OBAT'] for item in sublist]
treatment_counts = pd.Series(Counter(all_treatments))

# Display the results
print("Diagnosis Counts:\n", diagnosis_counts.sort_values(ascending=False))
# print("\nTreatment Counts:\n", treatment_counts.sort_values(ascending=False))


Diagnosis Counts:
 pulpitis reversible                       127
gingivitis marginalis kronis               91
pulpitis irreversible                      71
gingival abscess                           61
persistensi                                26
periodentitis                               3
impaksi                                     3
pulpitis akut                               2
48 impacted                                 2
gingivitis                                  2
gangren pulpa                               2
retained dental root                        1
gangren radix ( nekrosis )                  1
abs gingiva nekrosisi                       1
periodentitis apikalis akut                 1
impaksi gigi                                1
dental examination                          1
myalgia                                     1
gangren radi x                              1
fraktur lingual                             1
pulpitis irresversible                      1
pulpitis irvers

In [394]:
treatment_counts.sort_values(ascending=False)

scaling procedure                 97
evacuate gingiva                  52
t lc                              49
kaditic (2x1 dosage)              45
temporary seal                    43
                                  ..
pro ext ce                         1
mp 3x1                             1
asam mefenamat 1x1 prn             1
levo 2x1                           1
evaluasi 2 minggu lagi pro ext     1
Length: 193, dtype: int64

In [395]:

data = data.drop(columns=['DIAGNOSA', 'TERAPI OBAT'])

In [396]:
data = data.drop(columns=['NAMA KEPALA KELUARGA', 'TGL LAHIR'])
data

,NO,TANGGAL,NAMA PASIEN,ALAMAT LENGKAP,Nomor Hp,MEMBER,NON MEMBER,KELUHAN,UMUM/ GIGI,MEMBER FEE,NON MEMBER FEE,Member Remark,Age Category,Age,Gender,Cleaned DIAGNOSA,Cleaned TERAPI OBAT
0,1,2023-01-03 00:00:00,suparno,Desa Kanung 07/01,NaN,MEMBER,NaN,sakit gigi,P GIGI,NaN,Rp -,1,Dewasa (19-59 thn),58.0,L,[gingival abscess],"[clindamycin (3x1 dosage), piroxicam (2x1 dosa..."
1,2,2023-01-03 00:00:00,roikhan,Ds Kanung 05/02,82229436214,MEMBER,NaN,kontrol gigi,P GIGI,NaN,Rp -,1,Dewasa (19-59 thn),32.0,L,"[gingivitis marginalis kronis, pulpitis irreve...",[scaling procedure]
2,3,2022-01-05 00:00:00,ely ernawati,Desa rejosari RT/RW 02/01,NaN,NON MEMBER,NaN,scaling,P GIGI,NaN,60000,0,Dewasa (19-59 thn),34.0,P,[gingivitis marginalis kronis],[scaling procedure]
3,4,2023-01-09 00:00:00,deafi septina,Desa rejosari RT/RW 1/1,08581552550060,MEMBER,NaN,cabut gigi,P GIGI,NaN,Rp -,1,Anak (6-18 thn),15.0,P,[persistensi],[exo ce]
4,5,2023-01-12 00:00:00,siti amirul,Rejosari 7/2,081234377794,NaN,NON MEMBER,cabut gigi,P GIGI,NaN,90000,0,Dewasa (19-59 thn),40.0,P,"[pulpitis irversible, gingivitis marginalis kr...",[scaling procedure]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,506,2024-12-23 00:00:00,elvivana maisaroh,Ds Pucangrejo 13/06,081359274517,NaN,NON MEMBER,tambal gigi,GIGI,NaN,60000,0,Dewasa (19-59 thn),42.0,P,[pulpitis irreversible],[t lc]
506,507,2024-12-24 00:00:00,muhamad refan choirul iman,REJOSARI 07/02,NaN,NaN,NON MEMBER,lainnya,GIGI,NaN,30000,0,Anak (6-18 thn),6.0,L,[periodentitis],[ext topikal]
507,508,2024-12-24 00:00:00,narti,Ds. Kanung 02/02,NaN,MEMBER,NaN,kontrol gigi,GIGI,0.0,NaN,1,Dewasa (19-59 thn),58.0,P,[pulpitis irreversible],[t lc]
508,509,2024-12-24 00:00:00,dhinar anzani adevilyon,Karangmojo 13/4 Magetan,NaN,NaN,NON MEMBER,karang gigi,GIGI,NaN,0,0,Dewasa (19-59 thn),27.0,P,[gingivitis marginalis kronis],[scaling procedure]


In [397]:
# Count the number of missing values (NA) in the 'MEMBER FEE' column
data['MEMBER FEE'].isna().sum()


428

In [398]:
data['NON MEMBER FEE'].replace('Rp -', 0, inplace=True)
data['NON MEMBER FEE'].value_counts()

C:\Users\Zak\AppData\Local\Temp\ipykernel_20924\3067247517.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['NON MEMBER FEE'].replace('Rp -', 0, inplace=True)


NON MEMBER FEE
0         150
30000      30
20000      26
100000     24
25000      22
         ... 
18000       1
61000       1
47000       1
151000      1
36500       1
Name: count, Length: 62, dtype: int64

In [399]:
data['TANGGAL'] = pd.to_datetime(data['TANGGAL'], errors='coerce')

In [400]:
# replace - to 0 in 'MEMBER FEE' and 'NON MEMBER FEE'
data['MEMBER FEE'].replace('-', 0, inplace=True)
data['NON MEMBER FEE'].replace('-', 0, inplace=True)


C:\Users\Zak\AppData\Local\Temp\ipykernel_20924\613462583.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['MEMBER FEE'].replace('-', 0, inplace=True)
C:\Users\Zak\AppData\Local\Temp\ipykernel_20924\613462583.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [401]:
data['NON MEMBER FEE'] = data['NON MEMBER FEE'].astype(float)

In [402]:
data.to_csv('data_pasien.csv', index=False)

In [403]:
data

,NO,TANGGAL,NAMA PASIEN,ALAMAT LENGKAP,Nomor Hp,MEMBER,NON MEMBER,KELUHAN,UMUM/ GIGI,MEMBER FEE,NON MEMBER FEE,Member Remark,Age Category,Age,Gender,Cleaned DIAGNOSA,Cleaned TERAPI OBAT
0,1,2023-01-03,suparno,Desa Kanung 07/01,NaN,MEMBER,NaN,sakit gigi,P GIGI,NaN,0.0,1,Dewasa (19-59 thn),58.0,L,[gingival abscess],"[clindamycin (3x1 dosage), piroxicam (2x1 dosa..."
1,2,2023-01-03,roikhan,Ds Kanung 05/02,82229436214,MEMBER,NaN,kontrol gigi,P GIGI,NaN,0.0,1,Dewasa (19-59 thn),32.0,L,"[gingivitis marginalis kronis, pulpitis irreve...",[scaling procedure]
2,3,2022-01-05,ely ernawati,Desa rejosari RT/RW 02/01,NaN,NON MEMBER,NaN,scaling,P GIGI,NaN,60000.0,0,Dewasa (19-59 thn),34.0,P,[gingivitis marginalis kronis],[scaling procedure]
3,4,2023-01-09,deafi septina,Desa rejosari RT/RW 1/1,08581552550060,MEMBER,NaN,cabut gigi,P GIGI,NaN,0.0,1,Anak (6-18 thn),15.0,P,[persistensi],[exo ce]
4,5,2023-01-12,siti amirul,Rejosari 7/2,081234377794,NaN,NON MEMBER,cabut gigi,P GIGI,NaN,90000.0,0,Dewasa (19-59 thn),40.0,P,"[pulpitis irversible, gingivitis marginalis kr...",[scaling procedure]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,506,2024-12-23,elvivana maisaroh,Ds Pucangrejo 13/06,081359274517,NaN,NON MEMBER,tambal gigi,GIGI,NaN,60000.0,0,Dewasa (19-59 thn),42.0,P,[pulpitis irreversible],[t lc]
506,507,2024-12-24,muhamad refan choirul iman,REJOSARI 07/02,NaN,NaN,NON MEMBER,lainnya,GIGI,NaN,30000.0,0,Anak (6-18 thn),6.0,L,[periodentitis],[ext topikal]
507,508,2024-12-24,narti,Ds. Kanung 02/02,NaN,MEMBER,NaN,kontrol gigi,GIGI,0.0,NaN,1,Dewasa (19-59 thn),58.0,P,[pulpitis irreversible],[t lc]
508,509,2024-12-24,dhinar anzani adevilyon,Karangmojo 13/4 Magetan,NaN,NaN,NON MEMBER,karang gigi,GIGI,NaN,0.0,0,Dewasa (19-59 thn),27.0,P,[gingivitis marginalis kronis],[scaling procedure]


In [404]:
data.dtypes

NO                              int64
TANGGAL                datetime64[ns]
NAMA PASIEN                    object
ALAMAT LENGKAP                 object
Nomor Hp                       object
MEMBER                         object
NON MEMBER                     object
KELUHAN                        object
UMUM/ GIGI                     object
MEMBER FEE                    float64
NON MEMBER FEE                float64
Member Remark                   Int64
Age Category                   object
Age                           float64
Gender                         object
Cleaned DIAGNOSA               object
Cleaned TERAPI OBAT            object
dtype: object